In [2]:
!pip install plotly==5.5.0

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, precision_score, f1_score,roc_curve, roc_auc_score

In [4]:
data = pd.read_csv('affair_data.csv')
data.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affair
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1


In [5]:
X = data[data.columns[:-1]]
Y = data['affair']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [6]:
model = LogisticRegression()
model.fit(X_train, Y_train)
predictions = model.predict_proba(X_test)

In [7]:
roc_auc_score( Y_test, predictions[:, 1] )

0.7451629176618145

1910

In [8]:
np.linspace(0,1,101)

array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
       0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21,
       0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32,
       0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43,
       0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54,
       0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65,
       0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76,
       0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87,
       0.88, 0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98,
       0.99, 1.  ])

In [97]:
class lazy_roc_auc:
    import numpy as np
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots

    _thresholds = np.linspace(0, 1, 1001)
    __y_true =[]
    __y_scores =[]
    __ncalculated = True
    __tps = np.array([]) # True positives
    __fps = np.array([]) # False positives
    __fns = np.array([]) # False negatives
    __tns = np.array([]) # True negatives
    __tprs = np.array([]) # True Positive Rate
    __fprs = np.array([]) # True Positive Rate

    def __calc_confusion_matrix(self, threshhold):
        tp = 0 # True positive
        fp = 0 # False positive
        fn = 0 # False negative
        tn = 0 # True negative

        for predicted_prob, actual in zip( self.__y_scores[:,1], self.__y_true ):    
            if predicted_prob >= threshhold:
                predicted = 1
            else:
                predicted = 0
                
            if predicted == 1:
                if actual == 1:
                    tp += 1
                else:
                    fp += 1
            else:
                if actual == 1:
                    fn += 1
                else:
                    tn += 1
        return {'tp':tp, 'fp':fp, 'fn':fn, 'tn':tn}
    
    def __calc_basics(self):
        for i in range(self._thresholds.shape[0]):
            result = self.__calc_confusion_matrix(self._thresholds[i])
            self.__tps = self.np.append(self.__tps, result['tp'])
            self.__fps = self.np.append(self.__fps, result['fp'])
            self.__fns = self.np.append(self.__fns, result['fn'])
            self.__tns = self.np.append(self.__tns, result['tn'])

    def __calc_TPR(self):
        self.__tprs = self.__tps / (self.__tps + self.__fns)

    def __calc_FPR(self):
        self.__fprs = self.__fps / (self.__fps + self.__tns)

    def __calc_metrics(self):
        self.__calc_TPR()
        self.__calc_FPR()

    def __calculate(self):
        self.__calc_basics()
        self.__calc_metrics()
        self.__ncalculated = False

    def fit(self, y_true,y_scores):
        y_scores = self.np.array(y_scores)
        if(len(y_scores.shape) == 2):
            if(y_scores.shape[1] == 2):
                self.__y_true = self.np.array(y_true)
                self.__y_scores = y_scores
                self.__calculate()
                return
        print('Количество колонок в масиве с предсказаниями меньше или больше двух. \nДанная библиотека расчитана на бинарную класификацию.')

    def __check_data(self, y_true,y_scores):
        if ( (len(y_true) != 0) | (len(y_scores) != 0) ):
            if ( (len(y_true) == 0) | (len(y_scores) == 0) ):
                print('Ошибка в данных. Необходимо заполнить два параметра (y_true, y_scores) или воспользоваться методом fit().')
            else:
                self.fit(y_true, y_scores)      
    
    def plot(self,y_true = '', y_scores=''):
        self.__check_data(y_true, y_scores)
        if(self.__ncalculated):
            return
        fig  = make_subplots(cols=3, rows=1)
        fig.add_trace(
            self.go.Scatter(
                x=self.__fprs,
                y=self.__tprs,
                mode='lines',
                showlegend=False,
                fill='tozeroy'
            ),
            row = 1,
            col = 1
        )
        fig.add_trace(
            go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines',
                showlegend=False,
                line = dict(color='gray', width=1, dash='dash')
            ),
            row = 1,
            col = 1)
        fig.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across", spikethickness=1)
        fig.update_yaxes(showspikes=True, spikecolor="orange", spikesnap="cursor", spikethickness=1)
        #fig.update_layout(spikedistance=1000, hoverdistance=100)
        fig.update_layout(spikedistance=1000, )
        fig.show()

    def __init__(self,y_true = '', y_scores=''):
        self.__check_data(y_true, y_scores)
        return 

In [98]:
score = lazy_roc_auc(Y_test, predictions )
score.plot()

In [ ]:
        fig  = self.make_subplots(cols=2, rows=1)
        fig.add_trace(
            self.go.Scatter(
                x=self.__fprs,
                y=self.__tprs,
                mode='lines',
                showlegend=False,
                fill='tozeroy'
            ),
            row = 1,
            col = 1
        )
        fig.add_trace(
            self.go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines',
                showlegend=False,
                line = dict(color='gray', width=1, dash='dash')
            ),
            row = 1,
            col = 2
        )
        fig.update_xaxes(showspikes=True, spikecolor="green", spikesnap="cursor", spikemode="across", spikethickness=1)
        fig.update_yaxes(showspikes=True, spikecolor="orange", spikesnap="cursor", spikethickness=1)
        #fig.update_layout(spikedistance=1000, hoverdistance=100)
        fig.update_layout(spikedistance=10000, hoverdistance=10000)
        fig.show()

In [74]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
fig  = make_subplots(cols=2, rows=1)
fig.add_trace(
            go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode='lines',
                showlegend=False,
                line = dict(color='gray', width=1, dash='dash')
            ),
            row = 1,
            col = 2
)
fig.show()

In [12]:
tp = 0 # True positive
fp = 0 # False positive
fn = 0 # False negative
tn = 0 # True negative

predictions = model.predict_proba( X_test )
for predicted_prob, actual in zip( predictions[:, 1], Y_test ):    
    if predicted_prob >= 0.5:
        predicted = 1
    else:
        predicted = 0
    
    if predicted == 1:
        if actual == 1:
            tp += 1
        else:
            fp += 1
    
    else:
        if actual == 1:
            fn += 1
        else:
            tn += 1